In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time
import os
import cv2
from datasets import dataset_utils

# Main slim library
from tensorflow.contrib import slim

In [3]:
dataset_path = "C:\\Users\\samsung\\Day4\\data"
checkpoint_path = "C:\\Users\\samsung\\Day4\\data"
train_dir = './inception_finetuned/'

# Convert images into TFrecord files

In [4]:
from datasets import download_and_convert_flowers

In [5]:
download_and_convert_flowers.run(dataset_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\samsung\\Day4\\data/flower_photos'

# Dataset pipeline

In [ ]:
from preprocessing import inception_preprocessing
import tensorflow as tf

from tensorflow.contrib import slim


def load_batch(dataset, batch_size=32, height=299, width=299, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])
    
    # Preprocess image for usage by Inception.
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels = tf.train.batch(
          [image, image_raw, label],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size)
    
    return images, images_raw, labels

In [ ]:
path = '/home/dan/prj/datasets'

In [ ]:
def _get_filenames_and_classes(dataset_dir):
  """Returns a list of filenames and inferred class names.

  Args:
    dataset_dir: A directory containing a set of subdirectories representing
      class names. Each subdirectory should contain PNG or JPG encoded images.

  Returns:
    A list of image file paths, relative to `dataset_dir` and the list of
    subdirectories, representing class names.
  """
  flower_root = os.path.join(dataset_dir, 'flower_photos')
  directories = []
  class_names = []
  for dir_name in os.listdir(flower_root):
    path = os.path.join(flower_root, dir_name)
    if os.path.isdir(path):
      directories.append(path)
      class_names.append(dir_name)

  photo_filenames = []
  for directory in directories:
    for filename in os.listdir(directory):
      path = os.path.join(directory, filename)
      photo_filenames.append(path)

  return photo_filenames, sorted(class_names)

In [ ]:
import random

# Divide into train and test:
random.seed(0)
_NUM_VALIDATION = 350

random.shuffle(photo_filenames)
training_filenames = photo_filenames[_NUM_VALIDATION:]
validation_filenames = photo_filenames[:_NUM_VALIDATION]

In [ ]:
photo_filenames, class_names = _get_filenames_and_classes(path)

In [ ]:
class_names_to_ids = dict(zip(class_names, range(len(class_names))))

>>> def createGenerator():
...    mylist = range(3)
...    for i in mylist:
...        yield i*i
...
>>> mygenerator = createGenerator() # create a generator
>>> print(mygenerator) # mygenerator is an object!
<generator object createGenerator at 0xb7555c34>
>>> for i in mygenerator:
...     print(i)
0
1
4

In [ ]:
def createGenerator(image_path_list, split_name, class_names_to_ids):
    assert split_name in ['train', 'validation']
    for image_path in image_path_list:
        class_name = os.path.basename(os.path.dirname(image_path))
        class_id = class_names_to_ids[class_name]
        yield [cv2.imread(image_path), class_id]

## Efficient dataflow
### https://github.com/tensorpack/tensorpack/blob/master/docs/tutorial/efficient-dataflow.md
### https://github.com/tensorpack/tensorpack/blob/master/docs/tutorial/extend/dataflow.md
### https://github.com/tensorpack/tensorpack/issues/436


### http://openresearch.ai/t/tensorpack-multigpu/45
### https://github.com/ildoonet/tf-pose-estimation/blob/master/tf_pose/pose_dataset.py

In [ ]:
from tensorpack.dataflow import BatchData, DataFlow, TestDataSpeed

In [ ]:
class my_dataset_flow(DataFlow):
    def __init__(self, image_path_list, split_name, class_names_to_ids):
        assert split_name in ['train', 'validation']
        self.image_path_list = image_path_list
        self.split_name = split_name
        self.class_names_to_ids = class_names_to_ids
    def get_data(self):
        for image_path in self.image_path_list:
            class_name = os.path.basename(os.path.dirname(image_path))
            class_id = self.class_names_to_ids[class_name]
            yield [cv2.imread(image_path)]          
    def size(self):
        return len(self.image_path_list)

In [ ]:
train_dataset = my_dataset_flow(training_filenames, 'train', class_names_to_ids)
#validation_dataset = createGenerator(validation_filenames, 'validation')

In [ ]:
ds = BatchData(train_dataset, 256, use_list=True)
ds = PrefetchData(ds, 10, 2)

In [ ]:
TestDataSpeed(ds).start()

256 images/it * n it/s = 256 *n /s

In [ ]:
a = ds.get_data()

In [ ]:
for i in a:
    print(a.__class__)

In [9]:
from tensorpack.dataflow import BatchData, DataFlow, PrefetchData, PrefetchDataZMQ, TestDataSpeed
from tensorpack import QueueInput
from tensorpack import imgaug
import random
import numpy as np
import os


# reference
# https://github.com/tensorpack/tensorpack/blob/master/docs/tutorial/extend/dataflow.md
# https://github.com/tensorpack/tensorpack/blob/master/docs/tutorial/efficient-dataflow.md#ref
# https://github.com/tensorpack/tensorpack/blob/master/docs/tutorial/input-source.md
# http://openresearch.ai/t/tensorpack-multigpu/45
class my_dataset_flow(DataFlow):
    def __init__(self, image_path_list, split_name, class_names_to_ids):
        assert split_name in ['train', 'validation']
        self.image_path_list = image_path_list
        self.split_name = split_name
        self.class_names_to_ids = class_names_to_ids
    def get_data(self):
        for image_path in self.image_path_list:
            class_name = os.path.basename(os.path.dirname(image_path))
            class_id = self.class_names_to_ids[class_name]
            yield [cv2.resize(cv2.imread(image_path), (299, 299))]          
    def size(self):
        return len(self.image_path_list)


def createGenerator(image_path_list, split_name, class_names_to_ids):
    assert split_name in ['train', 'validation']
    for image_path in image_path_list:
        class_name = os.path.basename(os.path.dirname(image_path))
        class_id = class_names_to_ids[class_name]
        yield [cv2.imread(image_path), class_id]
def _get_filenames_and_classes(dataset_dir):
      """Returns a list of filenames and inferred class names.

      Args:
        dataset_dir: A directory containing a set of subdirectories representing
          class names. Each subdirectory should contain PNG or JPG encoded images.

      Returns:
        A list of image file paths, relative to `dataset_dir` and the list of
        subdirectories, representing class names.
      """
      flower_root = os.path.join(dataset_dir, 'flower_photos')
      directories = []
      class_names = []
      for dir_name in os.listdir(flower_root):
        path = os.path.join(flower_root, dir_name)
        if os.path.isdir(path):
          directories.append(path)
          class_names.append(dir_name)

      photo_filenames = []
      for directory in directories:
        for filename in os.listdir(directory):
          path = os.path.join(directory, filename)
          photo_filenames.append(path)

      return photo_filenames, sorted(class_names)
def main():
    print("Hello Tensorpack!")

    path = '/home/dan/prj/datasets'
    photo_filenames, class_names = _get_filenames_and_classes(path)

    random.seed(0)
    _NUM_VALIDATION = 350
    random.shuffle(photo_filenames)
    training_filenames = photo_filenames[:_NUM_VALIDATION]
    validataion_filenames = photo_filenames[_NUM_VALIDATION:]
    class_names_to_ids = dict(zip(class_names, range(len(class_names))))

    train_dataset = my_dataset_flow(training_filenames, 'train', class_names_to_ids)

    batchsize = 256
    nr_prefetch = 10
    nr_proc = 2
    
    ds = AugmentImageComponent(ds, [imgaug.Resize(299,299)])
    ds = PrefetchData(ds, 1000, multiprocessing.cpu_count())
    '''중요한 점은, 데이터를 읽는 부분이나 rotation, flip, crop 등의 augmentation을 정의하고 이를 PrefetchData에 넘기면 필요한 부분을 여러 프로세스로 띄워서 처리해준다는 점입니다.'''


    ds = BatchData(train_dataset, 256, use_list=True)
    ds = PrefetchDataZMQ(ds, 5)
    data = QueueInput(ds)

In [10]:
path = '/home/dan/prj/datasets'
photo_filenames, class_names = _get_filenames_and_classes(path)

In [11]:
random.seed(0)
_NUM_VALIDATION = 350
random.shuffle(photo_filenames)
training_filenames = photo_filenames[:_NUM_VALIDATION]
validataion_filenames = photo_filenames[_NUM_VALIDATION:]
class_names_to_ids = dict(zip(class_names, range(len(class_names))))

train_dataset = my_dataset_flow(training_filenames, 'train', class_names_to_ids)

batchsize = 256
nr_prefetch = 10
nr_proc = 2
#ds = AugmentImageComponent(ds, [imgaug.Resize(224)])
#ds = PrefetchData(ds, 1000, multiprocessing.cpu_count())
'''중요한 점은, 데이터를 읽는 부분이나 rotation, flip, crop 등의 augmentation을 정의하고 이를 PrefetchData에 넘기면 필요한 부분을 여러 프로세스로 띄워서 처리해준다는 점입니다.'''


ds = BatchData(train_dataset, 16, use_list=True)
ds = PrefetchData(ds, 10, 2)
data = QueueInput(ds)

[1022 23:43:03 @parallel.py:185] [MultiProcessPrefetchData] Will fork a dataflow more than one times. This assumes the datapoints are i.i.d.
[1022 23:43:03 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.
[1022 23:43:03 @argtools.py:152] WRN Install python-prctl so that processes can be cleaned with guarantee.


In [13]:
from tensorpack.input_source.input_source import EnqueueThread
sess = tf.Session()

placeholder = [tf.placeholder(dtype = tf.int8, shape=(batchsize, 299, 299, 3))]
queue = tf.FIFOQueue(50, [x.dtype for x in placeholder])
thread = EnqueueThread(queue, ds, placeholder)
tensors = queue.dequeue()

with sess.as_default():
    thread.start()
    for i in range(4):
        sess.run(tensors)

[1022 23:43:46 @input_source.py:151] ERR Exception in EnqueueThread fifo_queue_1:
Traceback (most recent call last):
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/tensorpack/input_source/input_source.py", line 144, in run
    self.op.run(feed_dict=feed)
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 2285, in run
    _run_using_default_session(self, feed_dict, self.graph, session)
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 4936, in _run_using_default_session
    session.run(operation, feed_dict)
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 905, in run
    run_metadata_ptr)
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 1113, in _run
    str(subfeed_t.get_shape())))
ValueError: Cannot feed value of shape (16, 299, 2

OutOfRangeError: FIFOQueue '_1_fifo_queue_1' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: fifo_queue_1_Dequeue = QueueDequeueV2[component_types=[DT_INT8], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](fifo_queue_1)]]

Caused by op 'fifo_queue_1_Dequeue', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/dan/.virtualenvs/tf1.6p3/lib/python3.5/site-packag

In [ ]:
from datasets import flowers
import tensorflow as tf

from tensorflow.contrib import slim

with tf.Graph().as_default(): 
    dataset = flowers.get_split('train', flowers_data_dir)
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32, common_queue_min=1)
    image, label = data_provider.get(['image', 'label'])
    
    with tf.Session() as sess:    
        with slim.queues.QueueRunners(sess):
            for i in range(4):
                np_image, np_label = sess.run([image, label])
                height, width, _ = np_image.shape
                class_name = name = dataset.labels_to_names[np_label]
                
                plt.figure()
                plt.imshow(np_image)
                plt.title('%s, %d x %d' % (name, height, width))
                plt.axis('off')
                plt.show()

# Training a CNN model

In [ ]:
def my_cnn(images, num_classes, is_training):  # is_training is not used...
    with slim.arg_scope([slim.max_pool2d], kernel_size=[3, 3], stride=2):
        net = slim.conv2d(images, 64, [5, 5])
        net = slim.max_pool2d(net)
        net = slim.conv2d(net, 64, [5, 5])
        net = slim.max_pool2d(net)
        net = slim.flatten(net)
        net = slim.fully_connected(net, 192)
        net = slim.fully_connected(net, num_classes, activation_fn=None)       
        return net

In [ ]:
from datasets import flowers

# This might take a few minutes.
train_dir = '/tmp/tfslim_model/'
print('Will save model to %s' % train_dir)

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)

    dataset = flowers.get_split('train', dataset_path)
    images, _, labels = load_batch(dataset)
  
    # Create the model:
    logits = my_cnn(images, num_classes=dataset.num_classes, is_training=True)
 
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)

    # Run the training:
    final_loss = slim.learning.train(
      train_op,
      logdir=train_dir,
      number_of_steps=1, # For speed, we just do 1 epoch
      save_summaries_secs=1)
  
    print('Finished training. Final batch loss %d' % final_loss)

# Fine-tune the model on a different set of labels.

In [ ]:
# Note that this may take several minutes.

import os

from datasets import flowers
from nets import inception
from preprocessing import inception_preprocessing

from tensorflow.contrib import slim
image_size = inception.inception_v1.default_image_size


def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["InceptionV1/Logits", "InceptionV1/AuxLogits"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)

    return slim.assign_from_checkpoint_fn(
      os.path.join(checkpoint_path, 'inception_v1.ckpt'),
      variables_to_restore)



with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = flowers.get_split('train', dataset_path)
    images, _, labels = load_batch(dataset, height=image_size, width=image_size)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes=dataset.num_classes, is_training=True)
        
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    final_loss = slim.learning.train(
        train_op,
        logdir=train_dir,
        init_fn=get_init_fn(),
        number_of_steps=100)
        
  
print('Finished training. Last batch loss %f' % final_loss)

# Evaluation

In [ ]:
import numpy as np
import tensorflow as tf
from datasets import flowers
from nets import inception

from tensorflow.contrib import slim

image_size = inception.inception_v1.default_image_size
batch_size = 10

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = flowers.get_split('train', dataset_path)
    images, images_raw, labels = load_batch(dataset, height=image_size, width=image_size)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes=dataset.num_classes, is_training=True)

    probabilities = tf.nn.softmax(logits)
    
    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())
    
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)
            np_probabilities, np_images_raw, np_labels = sess.run([probabilities, images_raw, labels])
    
            for i in range(batch_size): 
                image = np_images_raw[i, :, :, :]
                true_label = np_labels[i]
                predicted_label = np.argmax(np_probabilities[i, :])
                predicted_name = dataset.labels_to_names[predicted_label]
                true_name = dataset.labels_to_names[true_label]
                
                plt.figure()
                plt.imshow(image.astype(np.uint8))
                plt.title('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
                plt.axis('off')
                plt.show()